In [ ]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 14.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#write a convolutional neural network from unicornrecorder.csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mne
from sklearn.preprocessing import normalize


#import wandb
#from wandb.keras import WandbCallback

#wandb.init(project="my-test-project")


In [ ]:
#read in the data
neural_data = pd.read_csv('/content/drive/MyDrive/re-emote/finaldataset_withindex.csv',usecols=range(1,9))
#print(neural_data.shape)
#display(neural_data)
labels=df = open('/content/drive/MyDrive/re-emote/dataset/LabelList.txt', 'r')
scaler = StandardScaler()

In [ ]:



neural_image = scaler.fit_transform(neural_data)
print(tf.shape(neural_image))
#print(type(neural_data))
#neural_images=tf.reshape(neural_data, [-1,250,8])
#print(tf.shape(neural_images))

tf.Tensor([4091904       8], shape=(2,), dtype=int32)


In [ ]:
(len(neural_image))

NameError: ignored

In [ ]:
dataset=[]

In [ ]:
print(neural_image.shape)

(4091904, 8)


In [ ]:

neural_image.shape
neural_image=np.rollaxis(neural_image,1)
print(neural_image.shape)
finale=mne.time_frequency.stft(neural_image, 248, tstep=248/4, verbose=None)
real=np.abs(finale[:,0:100,:])
for i in range(len(finale[0,0,:])):
  normalized = normalize(real[:,:,i],axis=1)
  normalized=normalized.T
  dataset.append(normalized)

(8, 4091904)
Number of frequencies: 125
Number of time steps: 65999


In [ ]:
dataset=np.array(dataset)

In [ ]:
dataset.shape

(65999, 100, 8)

In [ ]:
labels=labels.read()
#print(labels)
list_labels=[]
for i in labels:
  try:
    int(i)
    list_labels.append(int(i))
  except:
    pass
labels=list_labels
labels=tf.keras.utils.to_categorical(np.array(labels)-1, num_classes = 4)
print(labels)
#print(labels[0],'lablesssrwg')
#print(type(labels))
print(len(labels))
print(labels)


In [ ]:
#let's print one image
plt.rcParams["figure.figsize"] = [20.00, 20.50]
plt.rcParams["figure.autolayout"] = True
data2D = dataset[:,:,0]

im = plt.imshow(data2D, cmap="nipy_spectral_r", origin='lower')


plt.colorbar(im)
plt.show()

TypeError: ignored

In [ ]:
#split the data into training and testing
aa=list(neural_images)
train_images, test_images,train_labels, test_labels = train_test_split(aa,labels, test_size=0.2,shuffle=True,random_state = 42)
#print(tf.convert_to_tensor(train_images).shape)
#print(type(train_labels))


train_labels=tf.convert_to_tensor(train_labels)
#print(train_labels)
#print(train_labels,'train labels')
#print(tf.shape(train_labels))


test_labels=tf.convert_to_tensor(test_labels)
#print(test_labels,'train label')
#print(tf.shape(test_labels))

train_features = tf.convert_to_tensor(train_images)
#print(type(train_features))
test_features = tf.convert_to_tensor(test_images)
#print(tf.shape(train_features),'fjkerogfnjjekognvdf')

In [ ]:
#build the convolutional model
model = keras.Sequential([
    layers.Conv2D(12, kernel_size=(12, 1), activation=tf.nn.gelu, input_shape=(125, 8,1),padding="same"),
    layers.AveragePooling2D(pool_size=(4, 1), strides=(4, 1), padding="same"),
    layers.Conv2D(250, kernel_size=(12, 1), activation=tf.nn.gelu,padding='same'),
    layers.AveragePooling2D(pool_size=(2, 1), strides=(2, 1),padding="same"),

    layers.Flatten(),
    layers.Dense(2000, activation=tf.nn.gelu),
    layers.Dropout(0.2),
    layers.Dense(1000, activation=tf.nn.gelu),
    layers.Dropout(0.2),
    layers.Dense(700, activation=tf.nn.gelu),
    layers.Dense(3, activation='softmax')

])


#compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

#train the model
history = model.fit(
    train_features, train_labels,
    validation_data=(test_features, test_labels),
    epochs=15,
    shuffle=True,
    #callbacks=[WandbCallback()]
)

#plot the results
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

plt.show()